In [ ]:
# Load and Split
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.documents import Document

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field

from textwrap import dedent
from operator import itemgetter
from pprint import pprint
import random

from dotenv import load_dotenv
load_dotenv()


In [2]:
COLLECTION_NAME = "olympic_info"
DOC_PATH = 'data/olympic.txt'
#  Vector DB와 동일하게 split(위의 것을 사용해도 된다.)
loader = TextLoader(DOC_PATH, encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", chunk_size=500, chunk_overlap=100
)
docs = loader.load_and_split(splitter) # list[Document]



In [ ]:
### 평가 데이터로 사용할 context 추출
total_samples = 3

# index shuffle 후 total_samples만큼 context 추출

idx_list = list(range(len(docs)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = docs[idx].page_content
    if len(context) > 100: # 100글자 이상인 text만 사용
        eval_context_list.append(context)

len(eval_context_list)

In [4]:
# user_input: 질문
# ####### retrieved_contexts: 검색된 문서의 내용(page_content)들
# qa_context: 질문 답변 쌍을 만들 때 참고한 context
        # retrieved_contexts: 검색된 문서의 내용은 실제 RAG 실행시 넣는다.
        # response: 모델의 답변 - 실제 RAG 실행시 넣는다.
# reference: 정답
class EvalDatasetSchema(BaseModel):
    user_input: str = Field(..., title="질문(question)")
    qa_context: list[str] = Field(..., title="질문-답변 쌍을 만들 때 참조한 context.")
    reference: str = Field(..., title="질문의 정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

eval_model = ChatOpenAI(model="gpt-4o")
prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개 질문-정답 쌍을 생성하세요. 
        
        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [Context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [Context]에 없는 내용을 가지고 질문-정답을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [Context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 정답을 만들고 그 내용이 [Context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어야 합니다.

        출력 형식: {format_instructions}
                    
        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
# print(prompt_template.template)

eval_dataset_generator = prompt_template | eval_model | parser

In [5]:
############################################################
# eval_context_list 모두로 만들기
# 
# 생성된 질문-답변을 눈으로 보고 검증한 및 수정해야 한다.
############################################################
eval_data_list = []
num_questions = 5
for context in eval_context_list:
    _eval_data_list = eval_dataset_generator.invoke({"context":context, "num_questions":num_questions})
    eval_data_list.extend(_eval_data_list)

In [ ]:
import pandas as pd
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

In [ ]:
## 생성 된 질문/답 쌍 확인
eval_df.head()
eval_df.tail()

# Chain 구성

In [14]:
# Vector Store 연결
COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/olympic_info"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [ ]:
vector_store._collection.count()

In [23]:
# Chain 구성
# prompt template. langchain hub에 등록된 것을 가져와서 사용.
prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':3,
        'fetch_k':10,
        'lambda_mult':0.5
    }
)
model = ChatOpenAI(model="gpt-4o-mini")

def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough() # rag chain을 RunnableSequence로 만들기 위해 Runnable인 것으로 시작.
    | {
        "context": retriever, "question":RunnablePassthrough()
    } # retriver -> {"context":list[Document], "question":"user input"}
    | {
        # 앞에서 넘어온 dictionary에서 context(List[Document])를 추출 -> page_content값들을 list로 반환. list[str]
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            # {"context":list[Document]} -> str(page_content들만 모은 string)
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()  # LLM 응답 처리 chain. 
    }
)

In [24]:
# rag_chain에 평가 질문을 입력해서 context들과 모델답변을 응답 받아 eval_dataset(eval_df)에 추가.
context_list = []
response_list = []

for user_input in eval_df['user_input']:
    res = rag_chain.invoke(user_input)
    context_list.append(res['source_context'])
    response_list.append(res['llm_answer'])

In [ ]:
print(len(context_list), len(response_list))
# pprint(context_list[:2])
# pprint(response_list[:2])

In [ ]:
eval_df["retrieved_contexts"] = context_list # context 추가
eval_df["response"] = response_list   # 정답 추가

In [ ]:
eval_df.head(3)

# 평가

In [ ]:
# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_dataset

In [ ]:
# model_name = "gpt-4o"
model_name = "gpt-4o-mini"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


## GPT-4o-mini 모델을 사용하여 평가 
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

In [ ]:
# gpt-4o : 분당 토큰 리미트에 걸려 RateLimitError 가 발생할 수있다. gpt-40: 30,000 TPM, gpt-4o-mini: 200,000 TPM
# 또한 network 연결등 문제가 발생하면 timeout 이 되어 평가가 실패할 수 있다.
# LLM 연결과 관련해 timetout이나 ratelimiterror 발생시 metrics를 나눠서 실행, 설정 변경을 통해 해결한다.
## https://platform.openai.com/settings/organization/limits
run_config = RunConfig(
    timeout=360,     # LLM 호출 이후 최대 대기 시간. 지정한 초까지 응답을 기다린다. 
    max_retries=20, # API 호출시 지정한 횟수만큼 재시도 한다.
    max_wait=360,   # 재시도 대기 시간(초) 180초 기다린 후 재시도 한다.
    max_workers=1   # 병렬처리 worker 수. 1로 설정하면 순차적으로 처리한다. (default: 16)
)
metrics1 = [
    LLMContextRecall(llm=eval_llm),
]
metrics2 = [
    LLMContextPrecisionWithReference(llm=eval_llm),
]
metrics3 = [
    Faithfulness(llm=eval_llm),
]
metrics4 = [
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]

In [ ]:
result1 = evaluate(dataset=eval_dataset, metrics=metrics1, run_config=run_config)
result1

In [ ]:
result2 = evaluate(dataset=eval_dataset, metrics=metrics2, run_config=run_config)
result2

In [ ]:
result3 = evaluate(dataset=eval_dataset, metrics=metrics3, run_config=run_config)
result3

In [ ]:
result4 = evaluate(dataset=eval_dataset, metrics=metrics4, run_config=run_config)
result4